In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pheochromocytoma_and_Paraganglioma"
cohort = "GSE64957"

# Input paths
in_trait_dir = "../../input/GEO/Pheochromocytoma_and_Paraganglioma"
in_cohort_dir = "../../input/GEO/Pheochromocytoma_and_Paraganglioma/GSE64957"

# Output paths
out_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/GSE64957.csv"
out_gene_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/gene_data/GSE64957.csv"
out_clinical_data_file = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/clinical_data/GSE64957.csv"
json_path = "../../output/preprocess/Pheochromocytoma_and_Paraganglioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Microarray study of human adrenal zona glomerulosa (ZG), zona fasciculata (ZF) and aldosterone-producing adenomas (APA)"
!Series_summary	"Learn about the transcriptome profiling of zona glomerulosa (ZG), zona fasciculata (ZF) and aldosterone-producing adenomas (APA) in human adrenals"
!Series_overall_design	"21 pairs of zona fasciculata (ZF) and zona glomerulosa (ZG), and 14 paired aldosterone-producing adenomas (APAs) from 14 Conn’s syndrome patients and 7 phaeochromocytoma patients were assayed on the Affymetrix Human Genome U133 Plus 2.0 Array. Laser capture microdissection was used to acquire samples of ZF, ZG and APA as previously described (Azizan EA, et al. J Clin Endocrinol Metab. 2012;97:E819-E829). For differentiation of ZG from ZF, sections were stained with cresyl violet using the LCM Staining Kit (AM1935, Ambion, USA). Data processing and analysis was performed using AffymetrixGeneChip Command Console Software and PartekGenomicSuite 6

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyzing data availability and type

# 1. Gene Expression Data Availability
# Looking at the background information, the data is mentioned as "transcriptome profiling" 
# using "Affymetrix Human Genome U133 Plus 2.0 Array", which is a gene expression array.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait (Pheochromocytoma), we see in the sample characteristics that disease information is in row 0
trait_row = 0  # Disease information is available in row 0

# Age information is not explicitly available in the sample characteristics
age_row = None  # Age data is not available

# Gender information is not available in the sample characteristics
gender_row = None  # Gender data is not available

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert disease trait values to binary format.
    0 = Not Pheochromocytoma (e.g., Conn's Syndrome)
    1 = Pheochromocytoma
    """
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after colon and strip whitespace
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the value contains "Pheochromocytoma"
    if "Pheochromocytoma" in value:
        return 1
    elif "Conn's Syndrome" in value:
        return 0
    else:
        return None

def convert_age(value):
    """
    Convert age values to continuous format.
    Since age data is not available, this function is a placeholder.
    """
    return None

def convert_gender(value):
    """
    Convert gender values to binary format.
    Since gender data is not available, this function is a placeholder.
    """
    return None

# 3. Save Metadata
# Determine trait availability
is_trait_available = trait_row is not None
# Call validate_and_save_cohort_info for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait data is available, extract clinical features
if trait_row is not None:
    # Create a clinical data DataFrame from the sample characteristics dictionary
    # First, create a dictionary to map sample characteristics rows to their values
    sample_chars = {0: ["disease: Conn's Syndrome", 'disease: Pheochromocytoma'], 
                    1: ['patient/tissue id: ADR106', 'patient/tissue id: ADR061', 'patient/tissue id: ADR031', 
                        'patient/tissue id: ADR026', 'patient/tissue id: ADR110', 'patient/tissue id: ADR111', 
                        'patient/tissue id: ADR017', 'patient/tissue id: ADR120', 'patient/tissue id: ADR115', 
                        'patient/tissue id: ADR085', 'patient/tissue id: ADR086', 'patient/tissue id: ADR074', 
                        'patient/tissue id: ADR032', 'patient/tissue id: ADR112', 'patient/tissue id: ADR113', 
                        'patient/tissue id: ADR121', 'patient/tissue id: ADR131', 'patient/tissue id: ADR044', 
                        'patient/tissue id: ADR118', 'patient/tissue id: ADR119', 'patient/tissue id: ADR116'], 
                    2: ['tissue: Human adrenal tissue'], 
                    3: ['tissue subtype: Zona Glomerulosa', 'tissue subtype: Zona Fasciculata', 
                        'tissue subtype: Aldosterone-Producing Adenoma']}
    
    # Extract sample IDs from the patient/tissue id field
    sample_ids = []
    for tissue_id in sample_chars[1]:
        if ':' in tissue_id:
            sample_id = tissue_id.split(':', 1)[1].strip()
            sample_ids.append(sample_id)
    
    # Create a DataFrame with sample IDs as columns
    clinical_data = pd.DataFrame(columns=sample_ids)
    
    # Add sample characteristics as rows
    for row_idx, values in sample_chars.items():
        for value in values:
            clinical_data.loc[row_idx] = value
    
    # Use geo_select_clinical_features to extract and process the clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical data
    preview_selected = preview_df(selected_clinical_df)
    print("Selected Clinical Data Preview:")
    print(preview_selected)
    
    # Save the selected clinical data to the output file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Selected Clinical Data Preview:
{'ADR106': [1.0], 'ADR061': [1.0], 'ADR031': [1.0], 'ADR026': [1.0], 'ADR110': [1.0], 'ADR111': [1.0], 'ADR017': [1.0], 'ADR120': [1.0], 'ADR115': [1.0], 'ADR085': [1.0], 'ADR086': [1.0], 'ADR074': [1.0], 'ADR032': [1.0], 'ADR112': [1.0], 'ADR113': [1.0], 'ADR121': [1.0], 'ADR131': [1.0], 'ADR044': [1.0], 'ADR118': [1.0], 'ADR119': [1.0], 'ADR116': [1.0]}
Clinical data saved to ../../output/preprocess/Pheochromocytoma_and_Paraganglioma/clinical_data/GSE64957.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be probe IDs from a microarray platform, not human gene symbols
# They are numeric values that don't match the pattern of human gene symbols (which are typically 
# alphanumeric and follow naming conventions like HUGO gene symbols)
# Therefore, these identifiers will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7892501', '7892502', '7892503', '7892504', '7892505'], 'GB_LIST': [nan, nan, nan, nan, nan], 'SPOT_ID': ['control', 'control', 'control', 'control', 'control'], 'seqname': [nan, nan, nan, nan, nan], 'RANGE_GB': [nan, nan, nan, nan, nan], 'RANGE_STRAND': [nan, nan, nan, nan, nan], 'RANGE_START': [nan, nan, nan, nan, nan], 'RANGE_STOP': [nan, nan, nan, nan, nan], 'probe_count': [4.0, 4.0, 4.0, 4.0, 4.0], 'transcript_cluster_id': [nan, nan, nan, nan, nan], 'exon_id': [nan, nan, nan, nan, nan], 'psr_id': [nan, nan, nan, nan, nan], 'gene_assignment': [nan, nan, nan, nan, nan], 'mrna_assignment': ['--- // pos_control // --- // --- // --- // ---', '--- // neg_control // --- // --- // --- // ---', '--- // neg_control // --- // --- // --- // ---', '--- // neg_control // --- // --- // --- // ---', '--- // neg_control // --- // --- // --- // ---'], 'probeset_type': ['normgene->exon', 'normgene->intron', 'normgene->intron', 'normgene->intron', 'normgene->intron']}

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for gene mapping
# Based on the preview, the 'ID' column appears to correspond to the probe identifiers
# For gene symbols, we need to extract them from 'gene_assignment' or 'mrna_assignment'
# Let's examine a few more rows to see what's in these columns

print("Checking gene assignment column content:")
gene_assignment_samples = gene_annotation['gene_assignment'].dropna().head(5)
print(gene_assignment_samples)

print("\nChecking mrna assignment column content:")
mrna_assignment_samples = gene_annotation['mrna_assignment'].dropna().head(5)
print(mrna_assignment_samples)

# 2. Get the gene mapping dataframe
# Based on observation, we need to extract gene symbols from the gene_assignment column
# Let's create a mapping dataframe with ID and gene symbols extracted from gene_assignment

# First, check if gene_assignment has content in later rows (first few rows were controls)
print("\nChecking more gene_assignment rows:")
more_gene_samples = gene_annotation.loc[gene_annotation['gene_assignment'].notna(), 'gene_assignment'].head(10)
print(more_gene_samples)

# Create the mapping dataframe
# It appears 'gene_assignment' contains gene information in the format:
# "Gene Symbol // Gene Title // Chromosomal Location // ..."
mapping_df = gene_annotation[['ID', 'gene_assignment']].copy()

# Clean and process gene_assignment to extract gene symbols
def extract_gene_symbol(gene_assignment_str):
    if pd.isna(gene_assignment_str):
        return None
    # Typically the gene symbol is the first part before //
    parts = gene_assignment_str.split(' // ')
    if parts and parts[0] != '---':
        return parts[0].strip()
    return None

mapping_df['Gene'] = mapping_df['gene_assignment'].apply(extract_gene_symbol)
mapping_df = mapping_df.dropna(subset=['Gene'])

print("\nPreview of mapping dataframe:")
print(preview_df(mapping_df[['ID', 'Gene']], n=5))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df[['ID', 'Gene']])

# Preview the gene expression data
print("\nPreview of gene expression data after mapping:")
print(preview_df(gene_data, n=5))


Checking gene assignment column content:
4201                                                  ---
4202                                                  ---
4203    NM_001005240 // OR4F17 /// NM_001004195 // OR4...
4204    NM_207366 // SEPT14 /// ENST00000388975 // SEP...
4205    NM_001005277 // OR4F16 /// NM_001005221 // OR4...
Name: gene_assignment, dtype: object

Checking mrna assignment column content:


0    --- // pos_control // --- // --- // --- // ---
1    --- // neg_control // --- // --- // --- // ---
2    --- // neg_control // --- // --- // --- // ---
3    --- // neg_control // --- // --- // --- // ---
4    --- // neg_control // --- // --- // --- // ---
Name: mrna_assignment, dtype: object

Checking more gene_assignment rows:
4201                                                  ---
4202                                                  ---
4203    NM_001005240 // OR4F17 /// NM_001004195 // OR4...
4204    NM_207366 // SEPT14 /// ENST00000388975 // SEP...
4205    NM_001005277 // OR4F16 /// NM_001005221 // OR4...
4206                                                  ---
4207                                                  ---
4208                                     AK172782 // GPAM
4209                                                  ---
4210    AK290103 // LOC100287934 /// ENST00000358533 /...
Name: gene_assignment, dtype: object



Preview of mapping dataframe:
{'ID': ['7896741', '7896743', '7896745', '7896751', '7896755'], 'Gene': ['NM_001005240', 'NM_207366', 'NM_001005277', 'AK172782', 'AK290103']}



Preview of gene expression data after mapping:
{'GSM1584943': [5.19322, 2.44778, 8.66997, 4.13934, 12.31021], 'GSM1584944': [5.45046, 3.41169, 8.03681, 3.18887, 12.07823], 'GSM1584945': [5.47963, 2.96345, 5.81757, 2.700708, 10.18567], 'GSM1584946': [5.08631, 2.84827, 4.33948, 3.6023, 7.43327], 'GSM1584947': [5.0255, 1.81085, 1.78552, 3.12501, 13.66089], 'GSM1584948': [4.99481, 1.34503, 5.21008, 4.69187, 8.881699999999999], 'GSM1584949': [4.38988, 2.7959, 3.90635, 4.5818900000000005, 10.816559999999999], 'GSM1584950': [4.5515, 1.30668, 6.37311, 2.573874, 12.23703], 'GSM1584951': [5.75597, 2.48221, 7.63714, 4.17232, 11.39821], 'GSM1584952': [4.66634, 2.03563, 7.0959, 3.5415900000000002, 13.247710000000001], 'GSM1584953': [4.08473, 2.80823, 7.1104, 3.312735, 13.44574], 'GSM1584954': [4.65597, 2.62894, 7.77417, 3.88112, 11.078420000000001], 'GSM1584955': [4.83007, 1.6533, 7.19232, 2.854, 9.5272], 'GSM1584956': [4.7537, 2.31225, 6.79486, 3.6642200000000003, 9.921710000000001], 'GSM1584957'

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was created in step 2
clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Clinical data shape: {clinical_data.shape}")
print(f"Clinical data sample: {clinical_data.head()}")

# Check if the clinical data needs to be transposed
# The read_csv might have loaded it with samples as columns, but we need samples as rows
if clinical_data.shape[0] == 1:  # If there's only one row (trait row)
    clinical_data = clinical_data.T
    print(f"Transposed clinical data shape: {clinical_data.shape}")
    
print(f"Clinical data sample IDs: {list(clinical_data.index)[:5]}...")
print(f"Gene data sample IDs: {list(normalized_gene_data.columns)[:5]}...")

# Check for sample ID overlap
clinical_samples = set(clinical_data.index)
gene_samples = set(normalized_gene_data.columns)
common_samples = clinical_samples.intersection(gene_samples)
print(f"Number of common samples between clinical and gene data: {len(common_samples)}")

# Filter data to include only common samples
if len(common_samples) > 0:
    clinical_data = clinical_data.loc[list(common_samples)]
    normalized_gene_data = normalized_gene_data[list(common_samples)]
    print(f"Filtered to {len(common_samples)} common samples")

    # Create linked data with samples as rows and features (trait + genes) as columns
    linked_data = pd.DataFrame(index=common_samples)
    linked_data[trait] = clinical_data.values  # Add the trait values
    
    # Add gene expression data (transposing so samples are rows)
    gene_data_for_samples = normalized_gene_data[list(common_samples)].T
    for gene in normalized_gene_data.index:
        linked_data[gene] = gene_data_for_samples[gene]
    
    print(f"Linked data shape: {linked_data.shape}")
    
    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    if linked_data.shape[0] > 0:
        is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)
    else:
        print("No data remains after handling missing values. Dataset cannot be used.")
        is_trait_biased = True
        unbiased_linked_data = linked_data
else:
    print("No common samples between clinical and gene data. Cannot proceed with linking.")
    linked_data = pd.DataFrame()
    is_trait_biased = True
    unbiased_linked_data = linked_data

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data for pheochromocytoma samples. Sample overlap between clinical and genetic data: {len(common_samples)}."
)

# 6. Save the data if it's usable
if is_usable and unbiased_linked_data.shape[0] > 0:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data saved to ../../output/preprocess/Pheochromocytoma_and_Paraganglioma/gene_data/GSE64957.csv
Clinical data shape: (1, 21)
Clinical data sample:                                     ADR106  ADR061  ADR031  ADR026  ADR110  \
Pheochromocytoma_and_Paraganglioma     1.0     1.0     1.0     1.0     1.0   

                                    ADR111  ADR017  ADR120  ADR115  ADR085  \
Pheochromocytoma_and_Paraganglioma     1.0     1.0     1.0     1.0     1.0   

                                    ...  ADR074  ADR032  ADR112  ADR113  \
Pheochromocytoma_and_Paraganglioma  ...     1.0     1.0     1.0     1.0   

                                    ADR121  ADR131  ADR044  ADR118  ADR119  \
Pheochromocytoma_and_Paraganglioma     1.0     1.0     1.0     1.0     1.0   

                                    ADR116  
Pheochromocytoma_and_Paraganglioma     1.0  

[1 rows x 21 columns]
Transposed clinical data shape: (21, 1)
Clinical data sample IDs: ['ADR106', 'ADR061', 'ADR031', 'ADR0